In [ ]:
!pip install sdv
import numpy as np
import pandas as pd
from sdv.demo import load_tabular_demo
from sdv.tabular import CTGAN
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_with_missing.csv to data_with_missing.csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['data_with_missing.csv']))
features_to_keep = ["company","level","title","location","yearsofexperience","yearsatcompany","gender","year","tag","tag_truncated","country"]
label =["basesalary"]
data = data[features_to_keep+label]
data.loc[data["level"].isin(data["level"].value_counts()[data["level"].value_counts()<=50].keys().tolist()),"level"]="other"
data.loc[data["tag"].isin(data["tag"].value_counts()[data["tag"].value_counts()<=50].keys().tolist()),"tag"]="other"
data["year"]=data["year"].apply(lambda x: x if pd.isnull(x) else x[:4])
data = data[data["basesalary"]!=0]
data = data[data["basesalary"]<=2*data["basesalary"].mean()]
train_data, test_data = train_test_split(data, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
test_data_dummy = pd.get_dummies(test_data, columns=["company","level","title","location","gender","tag_truncated","country","year"])


In [ ]:
test_data_dummy=test_data_dummy.drop("tag",axis=1)

In [ ]:
test_data_dummy["company_Booking.com"]=0

In [ ]:
ctgan_model=CTGAN()
ctgan_model.fit(train_data)

/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragm

In [ ]:
ctgan_model.save("ctgan_model")

In [ ]:
synthetic_data_ctgan = ctgan_model.sample(num_rows=50000)

In [ ]:
synthetic_data_ctgan.to_csv('synthetic_data_ctgan.csv', encoding = 'utf-8-sig') 
files.download('synthetic_data_ctgan.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
synthetic_data = ctgan_model.sample(num_rows=50000)

In [ ]:
synthetic_data = pd.get_dummies(synthetic_data, columns=["company","level","title","location","gender","tag_truncated","country","year"])

In [ ]:
synthetic_data=synthetic_data.drop("tag",axis=1)

In [ ]:
synthetic_data

,yearsofexperience,yearsatcompany,basesalary,company_AMD,company_Accenture,company_Adobe,company_Airbnb,company_Amazon,company_American Express,company_Apple,...,tag_truncated_others,country_Canada,country_Germany,country_US,country_United Kingdom,year_2017,year_2018,year_2019,year_2020,year_2021
0,0.09,0.976,151758.784555,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,13.31,5.474,147361.368027,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
2,12.68,0.954,125796.556780,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0.84,0.978,151500.101480,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,12.13,5.766,221133.037759,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,14.58,7.632,144473.114219,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
24996,5.05,1.005,82565.771324,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
24997,5.50,1.008,134889.353553,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
24998,11.68,0.996,80804.755323,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
X = synthetic_data.drop("basesalary", axis=1)
Y = synthetic_data["basesalary"]

regression = sm.OLS(Y, X).fit()
regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             basesalary   R-squared:                       0.438
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     73.49
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        09:02:03   Log-Likelihood:            -2.9839e+05
No. Observations:               25000   AIC:                         5.973e+05
Df Residuals:                   24737   BIC:                         5.994e+05
Df Model:                         262                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
yearsofexperience                             2863.8194     53.462     53.568      0.000    2759.031    2968.607
yearsatcompany                                2511.0350     84.520     29.709      0.000    2345.371    2676.699
company_AMD                                    321.9966   4174.356      0.077      0.939   -7859.991    8503.984
company_Accenture                            -1.261e+04   3045.839     -4.140      0.000   -1.86e+04   -6640.319
company_Adobe                                  558.6282   3177.909      0.176      0.860   -5670.264    6787.520
company_Airbnb                                8012.6695   3687.127      2.173      0.030     785.679    1.52e+04
company_Amazon                               -6577.0270   1171.538     -5.614      0.000   -8873.311   -4280.743
company_American Express                      5151.7023   5265.849      0.978      0.328   -5169.676    1.55e+04
company_Apple                                 5572.5575   1853.655      3.006      0.003    1939.283    9205.832
company_Atlassian                            -6274.1701   3691.820     -1.699      0.089   -1.35e+04     962.018
company_Autodesk                              5614.4967   4744.596      1.183      0.237   -3685.196    1.49e+04
company_Bloomberg                             3687.1077   1907.139      1.933      0.053     -50.999    7425.215
company_Boeing                                -528.2434   4808.615     -0.110      0.913   -9953.416    8896.929
company_Booking.com                          -4.317e+04   3.68e+04     -1.173      0.241   -1.15e+05     2.9e+04
company_Box                                   3092.8158   4757.050      0.650      0.516   -6231.286    1.24e+04
company_Broadcom                              7416.0629   3632.108      2.042      0.041     296.914    1.45e+04
company_ByteDance                            -1287.4297   4392.695     -0.293      0.769   -9897.375    7322.515
company_Capital One                          -3280.6591   2079.203     -1.578      0.115   -7356.021     794.703
company_Cisco                                 2278.6096   1708.302      1.334      0.182   -1069.764    5626.983
company_Comcast                              -2133.4480   7260.793     -0.294      0.769   -1.64e+04    1.21e+04
company_Cruise                                6807.9005   4738.061      1.437      0.151   -2478.984    1.61e+04
company_Dell Technologies                     2298.2677   3404.651      0.675      0.500   -4375.052    8971.587
company_Deloitte                             -7928.1765   2721.907     -2.913      0.004   -1.33e+04   -2593.077
company_DoorDash                              1777.1333   3229.070      0.550      0.582   -4552.037    8106.303
company_Dropbox                 

In [ ]:
predictions = regression.predict(test_data_dummy.drop(columns=["basesalary"]))
from sklearn.metrics import mean_squared_error
rmse = float(format(np.sqrt(mean_squared_error(test_data["basesalary"], predictions)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  47567.203


In [ ]:
from sdv.tabular import TVAE

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
clean_data = pd.read_csv(io.BytesIO(uploaded["cleaned_dataset.csv"]))

Saving cleaned_dataset.csv to cleaned_dataset.csv


In [ ]:
train_clean, test_clean = train_test_split(clean_data, test_size=0.2)

In [ ]:
tvae_model=TVAE()
tvae_model.fit(train_clean)

/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/ctgan/data_transformer.py:111

In [ ]:
tvae_model.save("tvae_model")

In [ ]:
synthetic_data_tvae = tvae_model.sample(num_rows=50000)

In [ ]:
synthetic_data_tvae.to_csv('synthetic_data_tvae.csv', encoding = 'utf-8-sig') 
files.download('synthetic_data_tvae.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
synthetic_data_tvae=synthetic_data_tvae.drop("Unnamed: 0",axis=1)

In [ ]:
x = synthetic_data_tvae.drop("basesalary", axis=1)
y = synthetic_data_tvae["basesalary"]

regression_1 = sm.OLS(y, x).fit()
regression_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             basesalary   R-squared (uncentered):                   0.974
Model:                            OLS   Adj. R-squared (uncentered):              0.973
Method:                 Least Squares   F-statistic:                              8479.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):                        0.00
Time:                        10:42:24   Log-Likelihood:                     -5.8108e+05
No. Observations:               50000   AIC:                                  1.163e+06
Df Residuals:                   49784   BIC:                                  1.164e+06
Df Model:                         216                                                  
Covariance Type:            nonrobust                                                  
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
yearsofexperience                             4803.0324     30.780    156.045      0.000    4742.704    4863.361
yearsatcompany                                 343.4927     49.459      6.945      0.000     246.553     440.432
company_AMD                                  -5861.1626   3168.033     -1.850      0.064   -1.21e+04     348.218
company_Accenture                            -1.011e+04   2431.675     -4.157      0.000   -1.49e+04   -5342.659
company_Adobe                                 -563.9530   1704.942     -0.331      0.741   -3905.659    2777.754
company_Airbnb                                 1.51e+04   2737.931      5.513      0.000    9729.121    2.05e+04
company_Amazon                               -1.102e+04    498.600    -22.096      0.000    -1.2e+04      -1e+04
company_American Express                     -1.485e+04   2598.828     -5.716      0.000   -1.99e+04   -9761.237
company_Apple                                 2449.6338    929.494      2.635      0.008     627.814    4271.453
company_Atlassian                             1184.1881   2448.251      0.484      0.629   -3614.413    5982.789
company_Autodesk                             -7601.6243   5537.881     -1.373      0.170   -1.85e+04    3252.688
company_Bloomberg                             1.892e+04   1675.475     11.292      0.000    1.56e+04    2.22e+04
company_Boeing                               -1.739e+04   6587.903     -2.640      0.008   -3.03e+04   -4476.556
company_Booking.com                          -2.292e-11   3.17e-11     -0.723      0.470   -8.51e-11    3.92e-11
company_Box                                   6166.8284   2588.036      2.383      0.017    1094.247    1.12e+04
company_Broadcom                              1952.4146   3691.287      0.529      0.597   -5282.552    9187.381
company_ByteDance                             1.541e+04   2557.777      6.027      0.000    1.04e+04    2.04e+04
company_Capital One                           -1.21e+04   1705.979     -7.092      0.000   -1.54e+04   -8754.626
company_Cisco                                -9388.1840   1042.802     -9.003      0.000   -1.14e+04   -7344.281
company_Comcast                               1.024e+04   8190.396      1.250      0.211   -5813.809    2.63e+04
company_Cruise                                1.704e+04   2344.148      7.271      0.000    1.25e+04    2.16e+04
company_Dell Technologies                    -4471.2560   1752.277     -2.552      0.011   -7905.739   -1036.773
company_Deloitte                             -1.216e+04   2808.960     -4.327      0.000   -1.77e+04   -6650.108
company_DoorDash                              

In [ ]:
test_clean=test_clean.drop("Unnamed: 0", axis=1)
predictions_1 = regression_1.predict(test_clean.drop("basesalary", axis=1))
from sklearn.metrics import mean_squared_error
rmse = float(format(np.sqrt(mean_squared_error(test_clean["basesalary"], predictions_1)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  30560.696


In [ ]:
x.columns

Index(['yearsofexperience', 'yearsatcompany', 'company_AMD',
       'company_Accenture', 'company_Adobe', 'company_Airbnb',
       'company_Amazon', 'company_American Express', 'company_Apple',
       'company_Atlassian',
       ...
       'tag_truncated_others', 'country_Canada', 'country_Germany',
       'country_US', 'country_United Kingdom', 'year_2017', 'year_2018',
       'year_2019', 'year_2020', 'year_2021'],
      dtype='object', length=266)

Synthetic data evaluation

In [ ]:
from sdv.metrics.tabular import NumericalLR

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
original_data = pd.read_csv(io.BytesIO(uploaded['cleaned_dataset.csv']))

Saving cleaned_dataset.csv to cleaned_dataset.csv


In [ ]:
from google.colab import files
uploaded_1 = files.upload()
import io
synthetic_data_tvae = pd.read_csv(io.BytesIO(uploaded_1['synthetic_data_tvae.csv']))

Saving synthetic_data_tvae.csv to synthetic_data_tvae.csv


In [ ]:
original_data=original_data.drop("Unnamed: 0", axis=1)

In [ ]:
synthetic_data_tvae=synthetic_data_tvae.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [ ]:
from google.colab import files
uploaded_2 = files.upload()
import io
real_data = pd.read_csv(io.BytesIO(uploaded_2['data_with_missing.csv']))
features_to_keep = ["company","level","title","location","yearsofexperience","yearsatcompany","gender","year","tag","tag_truncated","country"]
label =["basesalary"]
real_data = real_data[features_to_keep+label]
real_data.loc[real_data["level"].isin(real_data["level"].value_counts()[real_data["level"].value_counts()<=50].keys().tolist()),"level"]="other"
real_data.loc[real_data["tag"].isin(real_data["tag"].value_counts()[real_data["tag"].value_counts()<=50].keys().tolist()),"tag"]="other"
real_data["year"]=real_data["year"].apply(lambda x: x if pd.isnull(x) else x[:4])
real_data = real_data[real_data["basesalary"]!=0]
real_data = real_data[real_data["basesalary"]<=2*real_data["basesalary"].mean()]

Saving data_with_missing.csv to data_with_missing.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from google.colab import files
uploaded_3 = files.upload()
import io
synthetic_data_ctgan = pd.read_csv(io.BytesIO(uploaded_3['synthetic_data_ctgan.csv']))

Saving synthetic_data_ctgan.csv to synthetic_data_ctgan.csv


In [ ]:
real_data

,company,level,title,location,yearsofexperience,yearsatcompany,gender,year,tag,tag_truncated,country,basesalary
0,Oracle,L3,Product Manager,"Redwood City, CA",1.5,1.5,Unavailable,2017,NaN,NaN,US,107000.0
2,Amazon,L7,Product Manager,"Seattle, WA",8.0,0.0,Unavailable,2017,NaN,NaN,US,155000.0
3,Apple,M1,Software Engineering Manager,"Sunnyvale, CA",7.0,5.0,Unavailable,2017,NaN,NaN,US,157000.0
6,Microsoft,65,Software Engineering Manager,"Redmond, WA",15.0,11.0,Unavailable,2017,NaN,NaN,US,180000.0
7,Microsoft,62,Software Engineer,"Seattle, WA",4.0,4.0,Unavailable,2017,NaN,NaN,US,135000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35319,Amazon,L5,Software Engineer,"Seattle, WA",6.0,4.0,Unavailable,2018,Distributed Systems (Back-End),Distributed Systems (Back-End),US,147500.0
35320,eBay,other,Product Designer,"San Jose, CA",4.0,2.0,Unavailable,2018,Design,others,US,142000.0
35321,Google,T4,Software Engineer,"Seattle, WA",10.0,1.0,Unavailable,2018,Distributed Systems (Back-End),Distributed Systems (Back-End),US,155000.0
35322,Microsoft,62,Software Engineer,"Redmond, WA",2.0,2.0,Unavailable,2018,Full Stack,Full Stack,US,146900.0


In [ ]:
synthetic_data_ctgan=synthetic_data_ctgan.drop("Unnamed: 0", axis=1)

In [ ]:
NumericalLR.compute(original_data, synthetic_data_tvae, key_fields=["yearsofexperience","yearsatcompany"] ,sensitive_fields=["basesalary"]) 

/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
